In [ ]:
from describe import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#데이터 불러오기
all_data = pd.read_csv("data/all_data.csv")

In [ ]:
# target 값이 비어있지 않은 데이터(Train data)
train_data = all_data[all_data['target'].notnull()]
train_data = train_data.drop(['Set ID'], axis=1)    

# target 값이 비어있는 데이터(Test data)
test_data = all_data[all_data['target'].isnull()]

# 결과 확인
print(f"Train Data Shape: {train_data.shape}")
print(f"Test Data Shape: {test_data.shape}")

In [ ]:
# 각 열의 결측값 개수 확인
missing_values = train_data.isnull().sum()

# 결측값 비율 계산
missing_percentage = (missing_values / len(train_data)) * 100

# 결과를 데이터프레임으로 변환하여 보기 편하게 출력
missing_info = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage})
print(missing_info[missing_info['Missing Values'] > 0])

In [ ]:
# train_data에서 결측값이 있는 행의 개수 확인
missing_rows_count = train_data.isnull().any(axis=1).sum()

print(f"결측값이 있는 행의 개수: {missing_rows_count}")

In [ ]:
# 결측값이 있는 행 삭제
train_data_cleaned = train_data.dropna()

# 결과 확인
print(f"원래 Train Data Shape: {train_data.shape}")
print(f"클린된 Train Data Shape: {train_data_cleaned.shape}")


"""
결측값 처리 결과
    결측값 발생 행의 개수는 1개였으므로 제거하였다.
    결측값이 있는 행을 제거한 후의 Train Data Shape는 (40505, 60)이다.(Set ID 열도 제거하였음)
"""


In [ ]:
y = train_data_cleaned['target']
train_data_cleaned = train_data_cleaned.drop(['target'], axis=1)

범주형 데이터 인코딩 - 정규화

In [ ]:
### 1단계: 범주형 데이터 인코딩
from sklearn.preprocessing import LabelEncoder

# 범주형 변수를 인코딩하는 함수
def encode_categorical_features(data):
    label_encoders = {}
    for column in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))  # 문자열로 변환 후 인코딩
        label_encoders[column] = le
    return data, label_encoders

# Train data 인코딩
train_data_encoded, label_encoders = encode_categorical_features(train_data_cleaned) #이후 Inversion Transform 메서드를 사용하기 위해 label_encoders를 저장



In [ ]:
train_data_encoded.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 정규화 함수
def normalize_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data.select_dtypes(include=[np.number]))  # 수치형 데이터만 정규화
    scaled_df = pd.DataFrame(scaled_data, columns=data.select_dtypes(include=[np.number]).columns, index=data.index)
    return scaled_df

# 정규화 진행
train_data_scaled = normalize_data(train_data_encoded)

# 원본 데이터와 결합 (인코딩된 범주형 데이터는 함께 포함)
train_data_final = pd.concat([train_data_scaled, train_data_encoded.select_dtypes(include=['object'])], axis=1)

In [ ]:
train_data_final.shape

In [ ]:
y.shape

In [ ]:
#train_data_final csv 파일로 저장
train_data_final.to_csv("data/train_data_final.csv", index=False)
y.to_csv("data/y.csv", index=False)